# SILENT POET

This project aims to generate Shakeshpere style pargraphs using GANS. The GAN will try to generate a random pargraph which can later be improved to take some basic requirements like story line in the input itself to generate paragraph with those features.

###  Preparing The Dataset
1. Get the NLTk corpus
2. Join the Sentences to make a string
3. Combine Multiple Sentences to Make a Pragraph fo desired length.

In [1]:
import pickle as pk
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
import nltk.corpus as corpus
g_sents=corpus.gutenberg.sents()
g_sents=[" ".join(i) for i in g_sents]
data=[".".join(g_sents[i:i+25]) for i in range(len(g_sents)-1)]

### Preprocessing the Dataset

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
MAX_SEQUENCE_LENGTH=500
with open('word_index.pickle','rb') as word_index_file:
    word_index=pk.load(word_index_file)
tokenizer=Tokenizer(lower=True,filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.num_words=10000
tokenizer.fit_on_texts([" ".join(list(word_index.keys()))])
sequences=tokenizer.texts_to_sequences(data)
word_index=tokenizer.word_index
data=pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

Using TensorFlow backend.


### Prepare Embedding Matrix

In [4]:
with open('embeddings_pickle.pickle','rb') as embeddings:
     embeddings_index=pk.load(embeddings)
EMBEDDING_DIM = 50
num_words = len(word_index.keys())
embedding_matrix = np.zeros((num_words+1, EMBEDDING_DIM))
embeddings_index={k.lower():embeddings_index[k] for k in embeddings_index}
for word, i in word_index.items():    
    embedding_vector = embeddings_index.get(word.lower())
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
del embeddings_index,sequences,word_index,g_sents


In [5]:
def get_embedded_inputs(data):
    return np.reshape(embedding_matrix[data[:,:]],[-1,500,50,1])

### Build Generator

In [30]:
import tensorflow as tf
import numpy as np
BATCH_SIZE=10
NOISE_SHAPE=[16,16]
init = tf.global_variables_initializer()
def generateRandomShape(length=256,batch_size=5000):    
    inp=np.random.normal(0,1,size=[batch_size,length])
    return inp

def build_generator(inp,batch_size=32,reuse=False):
    with tf.variable_scope('gen',reuse=False) as scope:
        if(reuse):
            scope.reuse_variables()
        inp_r=tf.reshape(inp,shape=[batch_size,NOISE_SHAPE[0],NOISE_SHAPE[1],1])

        #conv2d layer 1
        g_l1=tf.layers.conv2d(inp_r,filters=16,kernel_size=(1,1),strides=(1,1),padding="SAME",name="g_l1")
        g_up1=tf.image.resize_images(g_l1,size=(50,18),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

        #conv2d layer 2
        g_l2=tf.layers.conv2d(g_up1,filters=32,kernel_size=(2,2),strides=(1,1),padding="SAME",name="g_l2")
        g_up2=tf.image.resize_images(g_l2,size=(100,20),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

        #conv2d layer 3
        g_l3=tf.layers.conv2d(g_up2,filters=16,kernel_size=(3,3),strides=(1,1),padding="SAME",name="g_l3")
        g_up3=tf.image.resize_images(g_l3,size=(250,25),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

        #conv2d layer 4
        g_l4=tf.layers.conv2d(g_up3,filters=1,kernel_size=(3,3),strides=(1,1),padding="SAME",name="g_l4")
        g_up4=tf.image.resize_images(g_l4,size=(500,50),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

        return g_up4

### Build Descriminator

In [7]:
def build_descriminator(inputs,reuse=False):
    #conv1
    with tf.variable_scope('dis',reuse=False) as scope:
        if(reuse):
            scope.reuse_variables()
        d_1=tf.layers.conv2d(inputs,filters=32,kernel_size=(3,3),padding="SAME",name="d_l1")     
        d_2=tf.layers.max_pooling2d(d_1,pool_size=(2,2),strides=(1,1),name="d_mp1")
        d_3=tf.layers.conv2d(d_2,filters=16,kernel_size=(3,3),padding="SAME",name="d_l2")
        d_4=tf.layers.max_pooling2d(d_3,pool_size=(2,1),strides=(1,1),name="d_mp2")
        d_5=tf.layers.conv2d(d_4,filters=16,kernel_size=(3,3),padding="SAME",name="d_l3")
        d_6=tf.layers.max_pooling2d(d_5,pool_size=(2,2),strides=(1,1),name="d_mp3")
        d_7=tf.layers.flatten(d_6)
        d_8=tf.layers.dropout(d_7,rate=0.5)
        d_9=tf.layers.dense(d_8,10,name="d_d2",reuse=tf.AUTO_REUSE)
        d_10=tf.layers.dense(d_9,2,name="d_d3",reuse=tf.AUTO_REUSE)
        d_11=tf.nn.sigmoid(d_10)
        return d_11

In [ ]:
train_x=generateRandomShape(batch_size=BATCH_SIZE)
x_=tf.placeholder(dtype=tf.float32,shape=(None,256))
d_x_=tf.placeholder(shape=[None,500,50,1],dtype=tf.float32)  
a=build_generator(x_,batch_size=BATCH_SIZE)
d_=build_descriminator(d_x_)
with tf.Session() as sess:
    init=tf.global_variables_initializer()
    tf.get_variable_scope().reuse_variables()
    sess.run(init)
    generated_output=sess.run(a,{x_:train_x})  
    print(sess.run(d_,feed_dict={d_x_:generated_output}))

### Define Losses for Generator and Discriminator

### Prepare Tensorboard

In [43]:
batch_size=32
tf.reset_default_graph()
o_x_=get_embedded_inputs(data[:1000,:])
x_placeholder=tf.placeholder(shape=[None,500,50,1],dtype=tf.float32)
z_placeholder=tf.placeholder(shape=[None,256],dtype=tf.float32)
Dx=build_descriminator(x_placeholder)
Gz=build_generator(z_placeholder,batch_size=batch_size)
Dg=build_descriminator(Gz,reuse=True)
#d_o_loss=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(Dx, tf.ones_like(Dx)))

d_o_loss=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Dx, labels=tf.ones_like(Dx)))
d_f_loss=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Dg,labels=tf.zeros_like(Dg)))
g_loss=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Dg,labels=tf.ones_like(Dg)))
tvars = tf.trainable_variables()
sess=tf.Session()

d_vars = [var for var in tvars if 'dis' in var.name]
g_vars = [var for var in tvars if 'gen' in var.name]

print([v.name for v in d_vars])
print([v.name for v in g_vars])

d_trainer_fake = tf.train.AdamOptimizer(0.0003).minimize(d_f_loss, var_list=d_vars)
d_trainer_real = tf.train.AdamOptimizer(0.0003).minimize(d_o_loss, var_list=d_vars)

# Train the generator
g_trainer = tf.train.AdamOptimizer(0.0001).minimize(g_loss, var_list=g_vars)

tf.summary.scalar('Generator_loss', g_loss)
tf.summary.scalar('Discriminator_loss_real', d_o_loss)
tf.summary.scalar('Discriminator_loss_fake', d_f_loss)

images_for_tensorboard = build_generator(z_placeholder,reuse=True)
tf.summary.image('Generated_images', images_for_tensorboard, 5)
merged = tf.summary.merge_all()
logdir = "SilentPoet/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

['dis/d_l1/kernel:0', 'dis/d_l1/bias:0', 'dis/d_l2/kernel:0', 'dis/d_l2/bias:0', 'dis/d_l3/kernel:0', 'dis/d_l3/bias:0', 'dis/d_d2/kernel:0', 'dis/d_d2/bias:0', 'dis/d_d3/kernel:0', 'dis/d_d3/bias:0']
['gen/g_l1/kernel:0', 'gen/g_l1/bias:0', 'gen/g_l2/kernel:0', 'gen/g_l2/bias:0', 'gen/g_l3/kernel:0', 'gen/g_l3/bias:0', 'gen/g_l4/kernel:0', 'gen/g_l4/bias:0']


In [44]:

sess.run(tf.global_variables_initializer())
index=0
for i in range(300):
    if((index+1)*10>data.shape[0]):
        index=0
    z_batch=generateRandomShape(batch_size=batch_size,length=256)
    real_text_batch=get_embedded_inputs(data[index*10:(index+1)*10])
    _, __, dLossReal, dLossFake = sess.run([d_trainer_real, d_trainer_fake, d_o_loss, d_f_loss],feed_dict={
        x_placeholder:real_text_batch,z_placeholder:z_batch
    })
    if(i % 100 == 0):
        print("dLossReal:", dLossReal, "dLossFake:", dLossFake)
    index+=1
    

dLossReal: 0.497493 dLossFake: 0.957878
dLossReal: 0.313262 dLossFake: 0.693147
dLossReal: 0.313262 dLossFake: 0.693148


In [ ]:
index=0
for i in range(100000):
    #train discriminator
    z_batch=generateRandomShape(batch_size=batch_size,length=256)
    
    if((index+1)*10>data.shape[0]):
        index=0
    real_text_batch=get_embedded_inputs(data[index*10:(index+1)*10])
    _, __, dLossReal, dLossFake = sess.run([d_trainer_real, d_trainer_fake, d_o_loss, d_f_loss],feed_dict={
        x_placeholder:real_text_batch,z_placeholder:z_batch
    })
    #train generator
    z_batch=generateRandomShape(batch_size=batch_size,length=256)
    _ = sess.run(g_trainer, feed_dict={z_placeholder: z_batch})
    if i % 10 == 0:
        # Update TensorBoard with summary statistics
        z_batch = generateRandomShape(batch_size=batch_size,length=256)
        summary = sess.run(merged, {z_placeholder: z_batch, x_placeholder: real_text_batch})
        writer.add_summary(summary, i)
    if i % 100 == 0:
        # Every 100 iterations, show a generated image
        print("Iteration:", i, "at", datetime.datetime.now())
        z_batch = generateRandomShape(batch_size=batch_size,length=256)
        generated_images = build_generator(z_placeholder, 1,reuse=True)
    index+=1
    

Iteration: 0 at 2018-06-13 21:38:11.522762
Iteration: 100 at 2018-06-13 21:43:17.737762
